In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../datasets/transaction_history.csv")

In [3]:
mcc = pd.read_csv("../data/MCC codes.csv")

In [4]:
data = data.merge(mcc , on = 'originalMcc')

In [5]:
data.query('`Business group` == "Продаж в роздріб" ').Category.unique()

array(['Різноманітні продовольчі магазини – ринки, магазини',
       'Бакалійні магазини, супермаркети',
       'Різні товари загального призначення',
       'Магазини, що торгують зі зниженими цінами'], dtype=object)

In [6]:
import time

In [7]:
current_week = int(time.strftime('%w'))

### Hight Level Report 

In [8]:
custom_categories = {
    'MCC_Categories' :  [{'originalMcc' : '5812', 'custom_category': 'Харчування'},{'originalMcc' : '5499', 'custom_category': 'Харчування'},{'originalMcc' : '5411', 'custom_category': 'Харчування'}],
    'Description_Categories' : [{'description' : 'Петро С.', 'custom_category': 'Квартплата'} , {'description': '535129****2010', 'custom_category': 'Комунальний платіж'}]
}

In [9]:
mcc_categories = pd.DataFrame([category for category in custom_categories['MCC_Categories']])
mcc_categories['originalMcc'] = mcc_categories['originalMcc'].astype(int)
data_with_mcc_categories = data.merge(mcc_categories,on='originalMcc', how='left')
description_categories = pd.DataFrame([category for category in custom_categories['Description_Categories']])
data_full = data_with_mcc_categories.merge(description_categories, on='description', how='left')
data_full['custom_category_x']=  data_full['custom_category_x'].fillna(data_full['custom_category_y'])
data_full['custom_category_x']=  data_full['custom_category_x'].fillna(data_full['Business group'])
data_full  = data_full.rename(columns={'custom_category_x':'custom_category'})
del data_full['custom_category_y']

In [10]:
data_full.query('week_number == 1')

,Unnamed: 0,id,time,description,originalMcc,amount,receiptId,type,full_date,month_number,week_number,user_name,telegram_id,Category,Business group,custom_category
0,0,QybTdwmdg1UxQYorUw,1736090319,Glovo,5499,434,E8C4-AH3E-CEA7-H0E8,outcome,2025-01-05,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
1,1,X2L4UyGbf3mHQnmHCg,1736016367,Анатолій К.,4829,715,T0E9-MC6C-3P2P-A72C,outcome,2025-01-04,1,1,Artem,533452580,Грошові перекази,Фінанси,Фінанси
2,2,dqY1XO0j5LgB88i-DUL2,1735950685,Glovo,5499,291,NaN,outcome,2025-01-04,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
3,3,6cexPC-Cv8T6zid9SkR7vBBmR7M,1735950685,Скасування. Glovo,5499,288,NaN,income,2025-01-04,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
4,4,5fRWZpgp0bXROL-_vA,1735920286,Glovo,5499,521,1914-H328-TBHP-KPTP,outcome,2025-01-03,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
5,5,GGq88fUZiC3i_Vs1Mg,1735905497,EVA,5977,671,49PX-MK5B-H506-XAC1,outcome,2025-01-03,1,1,Artem,533452580,Магазини косметики,Різні магазини,Різні магазини
6,6,C1tllvRqX2g5TGayyw,1735905223,SIMI #4133,5499,91,HKMK-8EBA-KH2B-0B3A,outcome,2025-01-03,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
7,7,VPummWGNsv4JbpH2fg,1735904985,APTEKA 32,5912,40,0867-0536-A1X3-XAC1,outcome,2025-01-03,1,1,Artem,533452580,Аптеки,Різні магазини,Різні магазини
8,8,cMfiFaVGkVnuDvdzAw,1735839580,Cs.Money,5816,596,1HT3-9B20-H3X4-MTPE,outcome,2025-01-02,1,1,Artem,533452580,Цифрові товари – ігри,Онлайн-майданчики,Онлайн-майданчики
9,9,tbRybYch9KBGAzwosg,1735837767,Cs.Money,5816,2117,E4X1-M34H-7C5E-AB72,outcome,2025-01-02,1,1,Artem,533452580,Цифрові товари – ігри,Онлайн-майданчики,Онлайн-майданчики


In [15]:
data_full.query("week_number == 1").groupby( ['user_name', 'Business group', 'custom_category'] ).amount.sum().to_frame()

amount
user_name Business group    custom_category          
Artem     Онлайн-майданчики Онлайн-майданчики    2713
          Продаж в роздріб  Харчування           4479
          Різні магазини    Різні магазини       2082
                            Харчування            120
          Фінанси           Квартплата           7000
                            Фінанси             11177
Ira       Продаж в роздріб  Харчування             67
          Транспортування   Транспортування       217
          Фінанси           Фінанси              6541

In [23]:
Balances = pd.read_csv("../datasets/balances.csv")

### Weekly balance report

In [27]:
Weekly = data_full.query("week_number == 1").groupby( ['custom_category'] ).amount.sum().reset_index().merge(Balances.query('Period == "w"'))

In [34]:
display(Weekly)
for category in Weekly.custom_category.unique():

    category_df = Weekly.query("custom_category == @category")

    availible = category_df['Balance'] -category_df['amount']
    availible = availible[0]
    if availible > 0 : 
        balance_report_body = f"For category {category}. You have {availible}."

    else :
        balance_report_body = f"For categpry {category}. You exceeded the weekly limit."

balance_report_body

,custom_category,amount,Balance,Period
0,Харчування,4666,3000,w


'For categpry Харчування. You exceeded the weekly limit.'

In [33]:
data.groupby('Business group').id.nunique()

Business group
Комунальні послуги       8
Онлайн-майданчики        3
Постачальники послуг     1
Продаж в роздріб        49
Різні магазини          12
Транспортування         18
Фінанси                 28
Name: id, dtype: int64